In [5]:
using Revise

using BilevelTrajOpt
using ForwardDiff
using DiffResults
using Plots
using LaTeXStrings

In [2]:
# Problem taken from:
# Sinha, Ankur, Pekka Malo, and Kalyanmoy Deb. 
# "A review on bilevel optimization: from classical to evolutionary approaches and applications." 
# IEEE Transactions on Evolutionary Computation 22.2 (2018): 276-295.

α = 10.
β = .2

δl = 1.
δf = 2.
γl = .3
γf = 5.8
cl = 10.
cf = 1.

P = (ql,qf) -> α - β*(ql + qf)
Cl = ql -> δl*ql*ql + γl*ql + cl
Cf = qf -> δf*qf*qf + γf*qf + cf

#7 (generic function with 1 method)

In [3]:
# closed form solution

qlopt = (2. * (β + δf)*(α - γl)-β * (α - γf))/(4. * (β + δf)*(β + δl) - 2. * β^2)
qfopt = (α - γf)/(2. * (β + δf)) - (β * (α - γl) - (β^2 * (α - γf))/(2. * (β + δf)))/(4. * (β + δf) * (β + δl) - 2. * β^2)

display(qlopt)
display(qfopt)

3.99236641221374

0.7730742539902844

In [6]:
# solution using our bilevel solver

function F(ql)
    fqf = qf -> -(P(ql[1],qf[1])*qf[1] - Cf(qf[1]))
    hqf = qf -> [0.]
    gqf = qf -> -qf
    qf0 = [0.]
    λ0 = [0.]
    μ0 = [0.]
    qfsol,λsol,μsol = auglag_solve(qf0,λ0,μ0,fqf,hqf,gqf,in_place=false)
    
    J = -(P(ql[1],qfsol[1])*ql[1] - Cl(ql[1]))
    c = -ql
    
    vcat(J,c)
end

ql0 = [0.]
Fres = DiffResults.JacobianResult(F(ql0),ql0)
num_g = 1
function bilevel_prob(ql)
    ForwardDiff.jacobian!(Fres, F, ql)
    Fv = DiffResults.value(Fres)
    FJ = DiffResults.jacobian(Fres)
        
    J = Fv[1]
    c = Fv[2:1+num_g]

    gJ = FJ[1,:]
    gc = FJ[2:1+num_g,:]
    
    fail = false
    
    J, c, gJ, gc, fail
end

lb = [-1e19]
ub = [1e19]
options = Dict{String, Any}()
options["Derivative option"] = 1
options["Verify level"] = -1
options["Major optimality tolerance"] = 1e-3

qlsol, fsol, info = snopt(bilevel_prob, ql0, lb, ub, options)

display(info)
display(qlsol)

function compute_qfsol(ql)
    fqf = qf -> -(P(ql[1],qf[1])*qf[1] - Cf(qf[1]))
    hqf = qf -> [0.]
    gqf = qf -> -qf
    qf0 = [0.]
    λ0 = [0.]
    μ0 = [0.]
    qfsol,λsol,μsol = auglag_solve(qf0,λ0,μ0,fqf,hqf,gqf)
end

qfsol,λsol,μsol = compute_qfsol(qlsol)
display(qfsol)

"Finished successfully: optimality conditions satisfied"

1-element Array{Float64,1}:
 3.9923664122137463

1-element Array{Float64,1}:
 0.7730742539903015

  likely near /home/blandry/.julia/packages/IJulia/GIANC/src/kernel.jl:41
  likely near /home/blandry/.julia/packages/IJulia/GIANC/src/kernel.jl:41
  likely near /home/blandry/.julia/packages/IJulia/GIANC/src/kernel.jl:41
  likely near /home/blandry/.julia/packages/IJulia/GIANC/src/kernel.jl:41
  likely near /home/blandry/.julia/packages/IJulia/GIANC/src/kernel.jl:41


In [120]:
# generating plots
N = 50
with_legend = false

thickfont = font(24)
legendfont = font(30)
guidefont = font(40)

clibrary(:Plots)
gr(size=(800,500), html_output_format=:png, seriescolor=:magma);

for image_i = 1:6
    αs = range(5.,stop=5.,length=N)
    βs = range(1.,stop=1.,length=N)
    δls = range(1.,stop=1.,length=N)
    δfs = range(1.,stop=1.,length=N)
    γls = range(1.,stop=1.,length=N)
    γfs = range(1.,stop=1.,length=N)
    cls = range(1.,stop=1.,length=N)
    cfs = range(1.,stop=1.,length=N)

    if image_i == 1
        αs = range(1.,stop=10.,length=N)
        xdata = αs 
        xlabelstring = "\\alpha"
    end

    if image_i == 2
        βs = range(.1,stop=5.,length=N)
        xdata = βs
        xlabelstring = "\\beta"
    end

    if image_i == 3
        δls = range(1.,stop=10.,length=N)
        xdata = δls
        xlabelstring = "\\delta_l"
    end

    if image_i == 4
        δfs = range(1.,stop=10.,length=N)
        xdata = δfs
        xlabelstring = "\\delta_f"
    end

    if image_i == 5
        γls = range(1.,stop=5.,length=N)
        xdata = γls
        xlabelstring = "\\gamma_l"
    end
        
    if image_i == 6
        γfs = range(1.,stop=5.,length=N)
        xdata = γfs
        xlabelstring = "\\gamma_f"
    end
            
    qlsols = zeros(N)
    qlopts = zeros(N)

    for i = 1:N
        α = αs[i]
        β = βs[i]

        δl = δls[i]
        δf = δfs[i]
        γl = γls[i]
        γf = γfs[i]
        cl = cls[i]
        cf = cfs[i]

        P = (ql,qf) -> α - β*(ql + qf)
        Cl = ql -> δl*ql*ql + γl*ql + cl
        Cf = qf -> δf*qf*qf + γf*qf + cf

        qlopts[i] = max(0.,(2. * (β + δf)*(α - γl)-β * (α - γf))/(4. * (β + δf)*(β + δl) - 2. * β^2))    

        function F(ql)
            fqf = qf -> -(P(ql[1],qf[1])*qf[1] - Cf(qf[1]))
            hqf = qf -> [0.]
            gqf = qf -> -qf
            qf0 = [0.]
            λ0 = [0.]
            μ0 = [0.]
            qfsol,λsol,μsol = auglag_solve(qf0,λ0,μ0,fqf,hqf,gqf,in_place=false,num_fosteps=1,num_sosteps=3)

            J = -(P(ql[1],qfsol[1])*ql[1] - Cl(ql[1]))
            c = -ql

            vcat(J,c)
        end

        ql0 = [0.]
        Fres = DiffResults.JacobianResult(F(ql0),ql0)
        num_g = 1
        function bilevel_prob(ql)
            ForwardDiff.jacobian!(Fres, F, ql)
            Fv = DiffResults.value(Fres)
            FJ = DiffResults.jacobian(Fres)

            J = Fv[1]
            c = Fv[2:1+num_g]

            gJ = FJ[1,:]
            gc = FJ[2:1+num_g,:]

            fail = false

            J, c, gJ, gc, fail
        end

        lb = [-1e19]
        ub = [1e19]
        options = Dict{String, Any}()
        options["Derivative option"] = 1
        options["Verify level"] = -1
        options["Major optimality tolerance"] = 1e-4

        qlsol, fsol, info = snopt(bilevel_prob, ql0, lb, ub, options)
        qlsols[i] = qlsol[1]
    end

    plot(xdata,qlopts,label="closed form solution",marker=3,
         xlabel=latexstring(xlabelstring),
         ylabel=latexstring("q_l^*"),
         legend=with_legend,
         xtickfont=thickfont,ytickfont=thickfont,legendfont=legendfont,guidefont=guidefont,
         seriescolor = :firebrick)
    plot!(xdata,qlsols,label="bilevel solver solution",marker=3,
        legend=with_legend,xtickfont=thickfont,ytickfont=thickfont,legendfont=legendfont,guidefont=guidefont,
        seriescolor = :dodgerblue)
    
    if with_legend
        savefig(string("figures/bilevel/bilevel_legend_",image_i,".svg"))
    else
        savefig(string("figures/bilevel/bilevel_",image_i,".svg"))        
    end
end

┌ Warning: Deprecated syntax `implicit assignment to global variable `image_i``.
│ Use `global image_i` instead.
└ @ nothing none:0
┌ Warning: Deprecated syntax `implicit assignment to global variable `F``.
│ Use `global F` instead.
└ @ nothing none:0
┌ Warning: Deprecated syntax `implicit assignment to global variable `F``.
│ Use `global F` instead.
└ @ nothing none:0
┌ Warning: Deprecated syntax `implicit assignment to global variable `bilevel_prob``.
│ Use `global bilevel_prob` instead.
└ @ nothing none:0
┌ Warning: Deprecated syntax `implicit assignment to global variable `bilevel_prob``.
│ Use `global bilevel_prob` instead.
└ @ nothing none:0
┌ Warning: Loop variable `image_i` around In[120]:11 overwrites a variable in an enclosing scope. In the future the variable will be local to the loop instead.
└ @ nothing In[120]:11


In [ ]:
# run this in a shell
for f in *.svg; do inkscape -E "eps/${f%.*}.eps" "$f"; done